**Stage I**    

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
import tensorflow as tf

In [5]:
from tensorflow import keras

image conversion

In [7]:
def saveImage(obj, location):
    i=0
    for batch in tqdm(datagen.flow(obj, batch_size=1,
                          save_to_dir='/Users/noahvirani/Downloads/newData/'+(location), save_prefix='', save_format='jpeg')):
        i += 1
        if i > 1:
            break 

In [9]:
df = pd.read_csv('https://astro.uchicago.edu/~andrey/classes/a211/data/training_solutions_rev1.csv')
arr = [np.zeros((1,1))]*61578
path = '/data/andrey/gzoo/images_training_rev1/'




In [18]:
def append_ext(fn):
    return str(fn)+".jpg"

ntest = 10000 # number of images in the testing subset
tests = np.random.choice(range(len(df)), ntest, replace=False)
traindf = df.drop(index=tests) # define training sample by dropping test images
testdf = df.iloc[tests] # define test sample 
testdf["GalaxyID"] = testdf["GalaxyID"].apply(append_ext)
traindf["GalaxyID"] = traindf["GalaxyID"].apply(append_ext)



<ipython-input-18-42462cc50063>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf["GalaxyID"] = testdf["GalaxyID"].apply(append_ext)


In [21]:
datagen=keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.1)
ntpix = 234
cmode = 'grayscale'

testgen=keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

bs = 32 # batch size for training 
valid_generator=datagen.flow_from_dataframe(traindf, directory=path, x_col='GalaxyID',
                                            y_col=['Class1.1','Class1.2'], target_size=(ntpix,ntpix),
                                            color_mode=cmode, class_mode='raw',
                                            batch_size=bs, interpolation='bilinear',
                                            subset='validation', shuffle=False)

train_generator=datagen.flow_from_dataframe(traindf, directory=path, x_col='GalaxyID',
                                            y_col=['Class1.1','Class1.2'], target_size=(ntpix,ntpix),
                                            color_mode=cmode,class_mode='raw',
                                            batch_size=bs, interpolation='bilinear',
                                            subset='training', shuffle=False)

test_generator=testgen.flow_from_dataframe(testdf, directory=path, x_col='GalaxyID',
                                           y_col=['Class1.1','Class1.2'], target_size=(ntpix,ntpix),
                                            color_mode=cmode, class_mode='raw',
                                            batch_size=1, interpolation='bilinear', shuffle=False)

Found 5157 validated image filenames.
Found 46421 validated image filenames.
Found 10000 validated image filenames.


Training:

In [22]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img



Model

In [39]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(234, 234, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())  
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [40]:
 model.fit(train_generator,epochs=10,validation_data=valid_generator,
                alidation_freq=1,verbose=1)

Epoch 1/10
1451/1451 [==============================] - 424s 212ms/step - loss: 0.6733 - accuracy: 0.6721 - val_loss: 0.5992 - val_accuracy: 0.7898
Epoch 2/10
1451/1451 [==============================] - 151s 104ms/step - loss: 0.5902 - accuracy: 0.8002 - val_loss: 0.5812 - val_accuracy: 0.8137
Epoch 3/10
1451/1451 [==============================] - 151s 104ms/step - loss: 0.5713 - accuracy: 0.8250 - val_loss: 0.5878 - val_accuracy: 0.8001
Epoch 4/10
1451/1451 [==============================] - 151s 104ms/step - loss: 0.5550 - accuracy: 0.8522 - val_loss: 0.5823 - val_accuracy: 0.8218
Epoch 5/10
1451/1451 [==============================] - 151s 104ms/step - loss: 0.5448 - accuracy: 0.8640 - val_loss: 0.5845 - val_accuracy: 0.8115
Epoch 6/10
1451/1451 [==============================] - 150s 104ms/step - loss: 0.5310 - accuracy: 0.8875 - val_loss: 0.5932 - val_accuracy: 0.8084
Epoch 7/10
1451/1451 [==============================] - 150s 104ms/step - loss: 0.5256 - accuracy: 0.8983 - val_

In [46]:
score = model.evaluate(test_generator, return_dict=True)

10000/10000 [==============================] - 59s 6ms/step - loss: 0.6006 - accuracy: 0.7926


In [50]:
print('Test accuracy:', score['accuracy'])

Test accuracy: 0.7925999760627747


In [ ]:
import pandas as pd

df = pd.read_csv('https://astro.uchicago.edu/~andrey/classes/a211/data/training_solutions_rev1.csv')

In [ ]:
print("Column headings:")
print(df.columns)
print(df.shape)

In [ ]:
# print the first 10 GalaxyIDs 
print(df['GalaxyID'][:10])

In [ ]:
# print the first 10 GalaxyIDs, pell, pdisk 
for i in range(10):
    # Class 1.1 = probability that galaxy has "elliptical" morpgology
    # Class 1.2 = probability that galaxy has "disk" morphology
    # Class 1.3 = probability that object is a star, not used here, but
    # in principle can be used to clean sample. 
    print("%d6   %.6f   %.6f"%(df['GalaxyID'][i], df['Class1.1'][i], df['Class1.2'][i]))

In [ ]:
from PIL import Image
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

def zoo_img_collage(gal_id, nrow, ncol, path=None, savefig=None):
    """
    display a collage of images of galaxies from the Galaxy Zoo 
    located in the directory path
    """
    fig, axs = plt.subplots(nrow, ncol, figsize=(ncol, nrow))

    # Check that PIL is installed for jpg support
    if 'jpg' not in fig.canvas.get_supported_filetypes():
        raise ValueError("PIL required to load SDSS jpeg images")

    for _gid, ax in zip(gal_id, axs.flatten()):
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        
        outfile = path+str(_gid)+'.jpg'
        I = Image.open(outfile)
        ax.imshow(I, origin='lower')
        ax.set_aspect('auto')
        
    fig.subplots_adjust(hspace=0, wspace=0)
    if savefig != None:
        plt.savefig(savefig, bbox_inches='tight')
    plt.show()


In [ ]:
path = 'projects/neural_nets/galaxy_zoo/images_training_rev1/'

ngal = 25
nrow = 5; ncol = 5
gal_id = df['GalaxyID'][:ngal]

zoo_img_collage(gal_id, nrow, ncol, path=path, savefig=None)

In [ ]:
# displaying pixel values of an image 
# ID of the first galaxy
gal_id = df['GalaxyID'][0]
# path to the image
img_file = path+str(gal_id)+'.jpg'
# open image
im = Image.open(img_file)
# convert image to pixel values of R,G,B
pixels = np.asarray(im)
# this will show us that we deal with 424 x 424 pixel images with 3 color channels: R, G, B
print(np.shape(pixels))

Converting R, G, B image to grayscale. There are different ways to do such conversion, see <a href="https://www.johndcook.com/blog/2009/08/24/algorithms-convert-color-grayscale/">here</a> for example. 

In [ ]:
# using "luminosity" method
lum = 0.21 *pixels[:,:,0] + 0.72 * pixels[:,:,1] + 0.07 * pixels[:,:,2]

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax.imshow(lum, origin='lower', cmap='Greys')
ax.set_aspect('auto')
plt.show()

Using scikit-image to read images and display them. Check out <a href="https://github.com/scikit-image/skimage-tutorials">scikit-image tutorials</a>.

In [ ]:
from skimage.io import imread_collection

#your path 
col_dir = path+'100*.jpg'

#creating a collection with the available images
col = imread_collection(col_dir)

In [ ]:
print(np.shape(col))

In [ ]:
from skimage.io import imshow_collection

imshow_collection(col)